In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gbor.main import BoostedOrdinal
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import time
from sklearn.model_selection import cross_val_score, RepeatedKFold
from enum import Enum

#wine_red = pd.read_csv('../data/winequality-red.csv', sep = ';')
wine_red = pd.read_csv('../data/winequality-white.csv', sep = ';')
wine_red['quality'] = wine_red['quality'] - np.min(wine_red['quality'])

X, y = wine_red.drop(columns = ['quality']).to_numpy(), wine_red['quality'].to_numpy(dtype = 'int')

# old stuff
class LineSearchState(Enum):
    IS_FIRST = 1
    WAS_DOUBLED = 2
    WAS_HALVED = 3

def _update_thresh_dev_old(thresh, dthresh, lr, X, y, g):
    this_state = LineSearchState.IS_FIRST
    this_accept = BoostedOrdinal._try_thresh(thresh, thresh -lr * dthresh, X, y, g)
    while True:
        if this_state == LineSearchState.IS_FIRST:
            if this_accept:
                lr_proposed = 2.0 * lr
                this_state = LineSearchState.WAS_DOUBLED
            else:
                lr_proposed = 0.5 * lr
                this_state = LineSearchState.WAS_HALVED
        else:
            if (this_state == LineSearchState.WAS_DOUBLED) and this_accept:
                # state remains was_doubled
                lr = lr_proposed # update lr
                lr_proposed = 2 * lr # propose new lr by doubling
            elif (this_state == LineSearchState.WAS_HALVED) and (not this_accept):
                # state remains was_halved
                lr = lr_proposed # update lr
                lr_proposed = 0.5 * lr # propose new lr by halving
            elif (this_state == LineSearchState.WAS_DOUBLED) and (not this_accept):
                # don't update lr, break
                break
            elif (this_state == LineSearchState.WAS_HALVED) and this_accept:
                # update lr and break
                lr = lr_proposed
                break
            else:
                raise Exception('We should not have logically reached this branch!!!')
        
        # potential BUG: this doesn't seem perfect, we are still getting warnings which is likely triggered by this line
        this_accept = BoostedOrdinal._try_thresh(thresh - lr * dthresh, thresh -lr_proposed * dthresh, X, y, g)

    return (thresh - lr * dthresh, lr)


In [2]:
gbor = BoostedOrdinal(
    n_iter_no_change = 10, max_iter = 1000
    , base_learner = DecisionTreeRegressor(max_depth = 6)
    , lr_g = 1e-1
    , lr_theta = 5#e-6
    , validation_stratify = False
    , validation_fraction = 0.2
)
start_time = time.time()
gbor = gbor.fit(X, y)
end_time = time.time()
print('time: {} sec'.format(round(end_time - start_time, 1)))

time: 0.6 sec


In [3]:
gbor.path['lr_theta']

array([5.00000000e+00, 6.10351562e-04, 1.22070312e-03, 6.10351562e-04,
       6.10351562e-04, 6.10351562e-04, 6.10351562e-04, 6.10351562e-04,
       6.10351562e-04, 6.10351562e-04, 6.10351562e-04, 6.10351562e-04,
       1.22070312e-03, 6.10351562e-04, 6.10351562e-04, 6.10351562e-04,
       1.22070312e-03, 6.10351562e-04, 1.22070312e-03, 6.10351562e-04,
       1.22070312e-03, 6.10351562e-04, 1.22070312e-03, 6.10351562e-04,
       1.22070312e-03, 6.10351562e-04, 1.22070312e-03, 1.22070312e-03,
       6.10351562e-04, 6.10351562e-04, 2.44140625e-03, 6.10351562e-04,
       2.44140625e-03, 6.10351562e-04, 2.44140625e-03, 6.10351562e-04,
       1.22070312e-03, 1.22070312e-03, 6.10351562e-04, 1.22070312e-03,
       1.22070312e-03])